In [2]:
import importlib
import helpers
importlib.reload(helpers)

ModuleNotFoundError: No module named 'helpers'

In [ ]:
# caminho_arquivo = '/workspaces/projeto_ing/apis/app/src/temp/tempo_Rio de Janeiro_20240928164640.txt'
# arquivo_criptografado = '/workspaces/projeto_ing/tests/app/src/dados_criptografados.bin'

# df_original, chave, cipher_suite, dados_criptografados, caminho_cripto = cripto_dados(
#     caminho=caminho_arquivo, 
#     arquivocript=arquivo_criptografado, 
#     delimitador=';'
# )

# print(f"Chave de criptografia: {chave.decode()}")

In [4]:
#variavies

import os
import json
import sqlite3
import pandas as pd
import shutil
from datetime import datetime
import sys

sys.path.append("/../../utils/")
sys.path.append("/workspaces/projeto_ing/utils/")
from helpers import cripto_dados
from helpers import consult_sqlite
from helpers import conectar_sqlite
from helpers import ler_arquivo

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
data_igtao = datetime.now().strftime('%Y-%m-%d')

caminho_backup = "/workspaces/projeto_ing/tests/app/ambiente_test_ing/backup/"
ctrl_ing = "/workspaces/projeto_ing/tests/app/ambiente_test_ing/app/archiving_ctrl/arquivo_controle.txt"
landing = "/workspaces/projeto_ing/tests/app/ambiente_test_ing/app/src/landing/"
layouts = "/workspaces/projeto_ing/tests/app/ambiente_test_ing/app/src/layouts/"